# Reliability Engineering
### FT Chart Calculation

### Calculate the system reliability and unreliability from FT chart

$R_i\cdots$ reliability of an element or event<br>
$F_i\cdots$ unreliability of an element or event

### Formulas
#### In "series system" $R_s = \prod_i^n R_i$
> $R_s = R_1 \cdot R_2$<br>
$F_s = F_1 + F_2 - F_1 \cdot F_2$<br>

#### In "parallel system" $F_s = \prod_i^n F_i$
> $R_s = R_1 + R_2 - R_1 \cdot R_2$ <br>
$F_s = F_1 \cdot F_2$

In [47]:
from math import*

In [48]:
# create event class
class Event:
    """argument: event(unreliability "F")"""
    def __init__(self, unreliability):
        self._F = unreliability
        self._R = 1 - unreliability
    
    @property
    def F(self):
        return self._F
    
    @F.setter
    def F(self, unreliability):
        self._F = unreliability
        self._R = 1 - unreliability
        return
    
    @property
    def R(self):
        return self._R
    
    @R.setter
    def R(self, reliability):
        self._R = reliability
        self._F = 1 - reliability
        return
    
    # operator overload
    def __add__(self, other):
        R_new = self.R * other.R
        return Event(1 - R_new) # Event(unreliability)
     
    def __mul__(self, other):
        F_new = self.F * other.F
        return Event(F_new)
    
    def __str__(self):
        return "F = {F}, R = {R}".format(F=self.F, R=self.R)

#### Event classの使い方
Instanciate element events and just calculate.<br>
各要素の不信頼度Fを求めて、それを引数にEvent Objectをつくって
ブール代数表記すれば、そのまま計算してくれるよ。<br>
$"*"$ as AND<br> $"+"$ as OR
>`XA = Event(unreliability=0.1)`<br>
 `XB = Event(unreliability=0.2)`<br>
 `XC = Event(unreliability=0.3)`<br>
 `T = (XA * XB) * (XA + XC)`


Example 1

In [56]:
# define each element XA to XC
XA = Event(0.1) # unreliability is 0.1
XB = Event(0.2) # unreliability is 0.2
XC = Event(0.3) # unreliability is 0.3
print(XA)
print(XB)
print(XC)

F = 0.1, R = 0.9
F = 0.2, R = 0.8
F = 0.3, R = 0.7


In [57]:
A = XA * XB
B = XA + XC
T = A * B
print(T)

F = 0.007400000000000001, R = 0.9926


Example 2

In [51]:
# define each unreliability F of an element
FM = 1 - exp(-2e-6*10000)
FS = 1 - exp(-3e-6*10000)
FE = 1 - exp(-5e-7*10000)
FB = 1 - exp(-5e-7*10000)
FG = 1 - exp(-5e-6*10000)

In [52]:
# definte each event of an element
M = Event(FM)
S = Event(FS)
E = Event(FE)
B = Event(FB)
G = Event(FG)

In [53]:
# T is top event
T = M + (S + (E + (B + G)))
print(T)

F = 0.10416586470347167, R = 0.8958341352965283
